# Flow and transport problem - SPE10

In this checkpoint we investigate the transport problem where the advective field is computed with a Darcy model.

Before creating the grid we import NumPy, the SciPy sparse library and PorePy.

In [ ]:
import numpy as np
import porepy as pp
import scipy.sparse as sps
import matplotlib.pyplot as plt

import os 
cwd_folder = os.getcwd()
spe10_folder = cwd_folder + "/spe10/"

import sys; sys.path.insert(1, spe10_folder)

from spe10 import Spe10

We specify number of cells in each dimension and the physical size of the domain. Then we create a Cartesian grid and compute geometric properties such as face centers, cell volumes etc.

In [ ]:
selected_layers = 35 

# Define the class with the corresponding layer(s)
spe10 = Spe10(selected_layers)
# For simplicity we extract the grid form the class spe10
sd = spe10.sd

# Read the permeability associated to the given layer(s)
perm_folder = spe10_folder + "/perm/"
spe10.read_perm(perm_folder)
perm_dict = spe10.perm_as_dict()

We declare the data for the Darcy problem.

In [ ]:
# Permeability
perm = pp.SecondOrderTensor(kxx=perm_dict["kxx"], kyy=perm_dict["kyy"], kzz=perm_dict["kzz"])      
injection_rate = 1
pos_well = [192.024, 233.17199999999997]  #this is the position found with the optimization
#it will be kept fixed

# Boundary conditions: homogeneous neumann everywhere
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]
b_face_centers = sd.face_centers[:, b_faces]
labels = np.array(["neu"] * b_faces.size)
bc_val = np.zeros(sd.num_faces)
bc = pp.BoundaryCondition(sd, b_faces, labels)

# Collect all parameters in a dictionary
parameters = {"second_order_tensor": perm, "bc": bc, "bc_values": bc_val}

We now set the data for the Darcy problem

In [ ]:
flow_key = "flow"
flow_data = pp.initialize_default_data(sd, {}, flow_key, parameters)

We now solve the Darcy problem by using the MPFA scheme.

In [ ]:
# construct the lhr and rhs from the discretization of the diffusion operator
mpfa = pp.Mpfa(flow_key)
mpfa.discretize(sd, flow_data)
A, b = mpfa.assemble_matrix_rhs(sd, flow_data)

# Add the injection wells, all with the same injection rate
b_wells = np.zeros_like(b)
index_iwells = [
   0,
   spe10.full_shape[0] - 1,
   spe10.full_shape[0] * spe10.full_shape[1] - spe10.full_shape[0],
   spe10.full_shape[0] * spe10.full_shape[1] - 1,
   ]
b_wells[index_iwells] = injection_rate

# Add the production well by using a Lagrange multiplier, first we identify the cell
ij_well = np.floor( (np.asarray(pos_well) / spe10.spacing[:-1])).astype(int)
index_pwell = spe10.full_shape[0] * ij_well[1] + ij_well[0]
vect = np.zeros((sd.num_cells, 1))
vect[index_pwell] = 1

# Solve the linear system and compute the pressure by adding the constraint
A = sps.bmat([[A, vect], [vect.T, None]], format="csc")
b = np.append(b + b_wells, 0)
cell_p = sps.linalg.spsolve(A, b)[:-1]
    
# now data contains the discretization matrices build from MPFA
mat_discr = flow_data[pp.DISCRETIZATION_MATRICES][flow_key]

q = mat_discr["flux"] @ cell_p + mat_discr["bound_flux"] @ bc_val

And we export the corresponding solutions.

In [ ]:
# to export the flux                                                                                                   
mvem = pp.MVEM(flow_key)                                                                                      
mvem.discretize(sd, flow_data)                                                                                      
                                                                                                                       
# construct the P0 flux reconstruction                                                                                 
cell_q = mvem.project_flux(sd, q, flow_data)

save = pp.Exporter(sd, "sol_p", folder_name="transport")

data_to_export = [("kxx", np.log10(perm_dict["kxx"])), 
                  ("kyy", np.log10(perm_dict["kyy"])), 
                  ("kzz", np.log10(perm_dict["kzz"])),
                  ("cell_p", cell_p),
                  ("cell_q", cell_q)]
save.write_vtu(data_to_export)

We now consider the transport problem where now the advective field is the one computed from the Darcy problem. First we set the data.

In [ ]:
# Transport problem
transport_key = "transport"
delta_t = 500
num_steps = 200

# boundary conditions for the advection problem: no flow everywhere
labels = np.array(["neu"] * b_faces.size)
bc_val = np.zeros(sd.num_faces)
bc = pp.BoundaryCondition(sd, b_faces, labels)

parameters = {"darcy_flux": q, "bc": bc, "bc_values": bc_val}
transport_data = pp.initialize_default_data(sd, {}, transport_key, parameters)

As done in the other cases, we now construct the upwind matrix and the mass matrix.

In [ ]:
# create the upwind and mass matrices
upwind = pp.Upwind(transport_key)  #the problem is linear, upwind flux is fine (a bit diffusive)

# discretize and get the matrices
upwind.discretize(sd, transport_data)

U, b_upwind = upwind.assemble_matrix_rhs(sd, transport_data)

M = sps.csr_matrix(np.diag(sd.cell_volumes), shape=np.shape(U))


Finally, by using the implicit Euler we compute the concentration that is transported in the porous medium

In [ ]:
# Initial condition: we have zero concentration except for a small square. The position of this square
# is what we aim to find to match the curve measured at the well

initial_conc_pos = [48,600]  # [292,505] #[300,400]
L = 50

ini_cond = np.logical_and((np.abs(sd.cell_centers[0,:]-initial_conc_pos[0])<L/2),(np.abs(sd.cell_centers[1,:]-initial_conc_pos[1])<L/2))
c = np.zeros(sd.num_cells)
c[ini_cond] = 1

initial_mass = np.sum(c*sd.cell_volumes) #total mass. This is the max we can collect at the well over time

#export initial condition
save = pp.Exporter(sd, "sol_c", folder_name="transport")
save.write_vtu([("conc", c)], time_step=0)

# IE with LU factorization
S = M + delta_t * U

# this partis a bit tricky: at the production well I chose to enforce zero concentration or, if you want, 
# infinite storage volume. This is because flow and concentration flux should be able to exit the cell to mimic 
# a production well.

# like for pressure, this is done with a Lagrange multiplier, adding a row and a column
S = sps.bmat([[S, vect], [vect.T, None]], format="csc")

outflow = [] #this is to store the flow exiting from the well in time
#the matrix will not change so it is convenient to factorize it once and for all
lu = sps.linalg.splu(S.tocsc())

for i in np.arange(num_steps):
    b = M @ c + delta_t * b_upwind
    b=np.append(b,0)
    sol = lu.solve(b)
    c = sol[:-1]
    ll = sol[-1] #this is the Lagrange multiplier and represents the flux exiting from the "well"
    save.write_vtu([("conc", c)], time_step=(i+1)*delta_t)
    outflow.append(ll)
    
plt.plot(np.arange(num_steps),outflow)
# export the main pvd file
time = np.arange((num_steps+1))*delta_t
save.write_pvd(time)

# let's check: mass conservation

total_outflow = np.sum(outflow)
final_internal_mass = np.sum(c*sd.cell_volumes)

print('initial mass', initial_mass)
print('total outflow', total_outflow)
print('mass trapped in the domain', final_internal_mass)

print('error in mass conservation',initial_mass-total_outflow-final_internal_mass) #must be 0 or machine eps.

In [ ]:
def Checkpoint3_solution(selected_layers,initial_conc_pos, L=50):

    outflow = np.zeros((200,))
                       
    # surrogate model implementation

    return outflow

In [ ]:
selected_layers = [51, 71, 42, 20, 12, 2, 82]

if __name__ == '__main__':

    for i in range(7):
    
        outflow = Checkpoint3_solution(selected_layers[i], [120, 250] )